# Pathology training with Mask R-CNN
## Train with malignant and benign class instead of a single mass class

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import time
import concurrent.futures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug
from imgaug import augmenters as iaa

# Root directory of the project 
# must add to system path before import mrcnn
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
sys.path.append(ROOT_DIR)

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log

import mammo_baseline_pathology

%matplotlib inline 



/home/liujing/liuyibo/Mammo_MaskRCNN


Using TensorFlow backend.


In [2]:
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

## Configurations

In [3]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, "dataset/mammo_all")

# Load dataset
subset = "mass_train"
if "3x" in subset:
    augmented=True
else:
    augmented=False

dataset_train = mammo_baseline_pathology.MammoDataset()
dataset_train.load_mammo(DATASET_DIR, subset=subset, augmented=augmented, json_filename="mammo_all_ddsm_mass_train.json")

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

# Load validation dataset
dataset_val = mammo_baseline_pathology.MammoDataset() # May17
dataset_val.load_mammo(DATASET_DIR, "mass_val", augmented=augmented, json_filename="mammo_all_ddsm_mass_train.json")
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

mass_train
Image Count: 985
Class Count: 3
  0. BG                                                
  1. MALIGNANT                                         
  2. BENIGN                                            
mass_val
Images: 246
Classes: ['BG', 'MALIGNANT', 'BENIGN']


## Notebook Preferences

In [4]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [5]:
#######################################
#   Training with Pathology labelling #
#      images. Max_dim = 1024         # 
#      Training set size = 4x         # 
#######################################

# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_baseline_pathology.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 1 
    
    IMAGE_MAX_DIM = 1024

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + Malignant + Benign

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000 # Is it the number of ROIs?
    POST_NMS_ROIS_INFERENCE = 1000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [ ]:
# save model path
MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)

# load weights
weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
print("Loading weights:", weights_path)
model.load_weights(weights_path, by_name=True)
print("Weights loaded.")

# train
model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=80, layers='all')


Loading weights: resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
Weights loaded.

Starting at epoch 0. LR=0.001

Checkpoint Path: checkpoints/mammo_pathology20190520T0003/mask_rcnn_mammo_pathology_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a      

/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/80


ERROR:root:Error processing image {'id': 'Mass-Training_P_00076_LEFT_CC', 'source': 'pathology', 'path': '/home/liujing/liuyibo/Mammo_MaskRCNN/dataset/mammo_all/mass_train/Mass-Training_P_00076_LEFT_CC/full_image/Mass-Training_P_00076_LEFT_CC.png', 'pathology': ['BENIGN'], 'catID': [2]}
Traceback (most recent call last):
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/ImageFile.py", line 224, in load
    s = read(self.decodermaxblock)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 652, in load_read
    cid, pos, length = self.png.read()
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 122, in read
    length = i32(s)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/_binary.py", line 77, in i32be
    return unpack_from(">I", c, o)[0]
struct.error: unpack_from requires a buffer of at least 4 bytes

During handling of the above except

 89/985 [=>............................] - ETA: 17:30 - loss: 1.8467 - rpn_class_loss: 0.1741 - rpn_bbox_loss: 1.0922 - mrcnn_class_loss: 0.0308 - mrcnn_bbox_loss: 0.3123 - mrcnn_mask_loss: 0.2372

...

528/985 [===============>..............] - ETA: 4:58 - loss: 0.1652 - rpn_class_loss: 9.0272e-04 - rpn_bbox_loss: 0.0208 - mrcnn_class_loss: 0.0056 - mrcnn_bbox_loss: 0.0285 - mrcnn_mask_loss: 0.1094

ERROR:root:Error processing image {'id': 'Mass-Training_P_00076_LEFT_CC', 'source': 'pathology', 'path': '/home/liujing/liuyibo/Mammo_MaskRCNN/dataset/mammo_all/mass_train/Mass-Training_P_00076_LEFT_CC/full_image/Mass-Training_P_00076_LEFT_CC.png', 'pathology': ['BENIGN'], 'catID': [2]}
Traceback (most recent call last):
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/ImageFile.py", line 224, in load
    s = read(self.decodermaxblock)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 652, in load_read
    cid, pos, length = self.png.read()
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 122, in read
    length = i32(s)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/_binary.py", line 77, in i32be
    return unpack_from(">I", c, o)[0]
struct.error: unpack_from requires a buffer of at least 4 bytes

During handling of the above except

985/985 [==============================] - 738s 750ms/step - loss: 0.1659 - rpn_class_loss: 9.1892e-04 - rpn_bbox_loss: 0.0213 - mrcnn_class_loss: 0.0056 - mrcnn_bbox_loss: 0.0278 - mrcnn_mask_loss: 0.1102 - val_loss: 1.7487 - val_rpn_class_loss: 0.0076 - val_rpn_bbox_loss: 0.3828 - val_mrcnn_class_loss: 0.1242 - val_mrcnn_bbox_loss: 0.3206 - val_mrcnn_mask_loss: 0.9134
Epoch 53/80
747/985 [=====================>........] - ETA: 2:35 - loss: 0.1629 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0193 - mrcnn_class_loss: 0.0055 - mrcnn_bbox_loss: 0.0271 - mrcnn_mask_loss: 0.1100

ERROR:root:Error processing image {'id': 'Mass-Training_P_00076_LEFT_CC', 'source': 'pathology', 'path': '/home/liujing/liuyibo/Mammo_MaskRCNN/dataset/mammo_all/mass_train/Mass-Training_P_00076_LEFT_CC/full_image/Mass-Training_P_00076_LEFT_CC.png', 'pathology': ['BENIGN'], 'catID': [2]}
Traceback (most recent call last):
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/ImageFile.py", line 224, in load
    s = read(self.decodermaxblock)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 652, in load_read
    cid, pos, length = self.png.read()
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 122, in read
    length = i32(s)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/_binary.py", line 77, in i32be
    return unpack_from(">I", c, o)[0]
struct.error: unpack_from requires a buffer of at least 4 bytes

During handling of the above except

985/985 [==============================] - 740s 751ms/step - loss: 0.1630 - rpn_class_loss: 0.0010 - rpn_bbox_loss: 0.0188 - mrcnn_class_loss: 0.0055 - mrcnn_bbox_loss: 0.0265 - mrcnn_mask_loss: 0.1112 - val_loss: 1.7070 - val_rpn_class_loss: 0.0113 - val_rpn_bbox_loss: 0.4246 - val_mrcnn_class_loss: 0.1332 - val_mrcnn_bbox_loss: 0.3425 - val_mrcnn_mask_loss: 0.7955
Epoch 54/80
279/985 [=======>......................] - ETA: 7:40 - loss: 0.1658 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0199 - mrcnn_class_loss: 0.0072 - mrcnn_bbox_loss: 0.0251 - mrcnn_mask_loss: 0.1125

In [8]:
#plot loss 
history=model.keras_model.fit(steps_per_epoch=config.STEPS_PER_EPOCH, validation_steps=config.VALIDATION_STEPS, 
                              batch_size=config.BATCH_SIZE)
# loss     rpn_class_loss     rpn_bbox_loss     mrcnn_class_loss     mrcnn_bbox_loss     mrcnn_mask_loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train","test"],loc="upper left")
plt.show()

UnboundLocalError: local variable 'val_ins' referenced before assignment